### Import Required Libraries and Set Up Environment Variables

In [78]:
# Dependencies
import requests
import time
from dotenv import load_dotenv
import os
import pandas as pd
import json

In [79]:
# Set environment variables from the .env in the local environment
###Day 2 Activity 2###
load_dotenv(".env")

nyt_api_key = os.getenv("NYT_API_KEY")
tmdb_api_key = os.getenv("TMDB_API_KEY")

load_dotenv()

True

### Access the New York Times API

In [80]:
#######################################################################Day 2 Activity 6 for the next 2 cells####

# Set the base URL
url = "https://api.nytimes.com/svc/search/v2/articlesearch.json?"

###################################################################requires you to read the NYT documentation####
# Filter for movie reviews with "love" in the headline
# section_name should be "Movies"
# type_of_material should be "Review"
##############################################search for "Filter Query Examples" in the Article Search API docs.
filter_query = 'section_name:"Movies" AND type_of_material:"Review" AND headline:"love"'

# Use a sort filter, sort by newest
sort = "newest"

# Select the following fields to return:
# headline, web_url, snippet, source, keywords, pub_date, byline, word_count
###############################################look for "Limit Fields in Response in the Article Search API Docs"

field_list = "headline,web_url,snippet,source,keywords,pub_date,byline,word_count"

# Search for reviews published between a begin and end date
begin_date = "20130101"
end_date = "20230531"

# Build URL

query_url = (
    f"{url}api-key={nyt_api_key}&begin_date={begin_date}&end_date={end_date}"
    + f'&fq={filter_query}&sort={sort}&fl={field_list}'
)
query_url

'https://api.nytimes.com/svc/search/v2/articlesearch.json?api-key=7dLLnh8E7AcFN2yxNK69lfeef8gG13eQ&begin_date=20130101&end_date=20230531&fq=section_name:"Movies" AND type_of_material:"Review" AND headline:"love"&sort=newest&fl=headline,web_url,snippet,source,keywords,pub_date,byline,word_count'

In [81]:
# Create an empty list to store the reviews
reviews_list = []

# loop through pages 0-19
for page in range(0, 3):
    # create query with a page number
    query_url = f"{query_url}&page={str(page)}"
    # API results show 10 articles at a time
    # Make a "GET" request and retrieve the JSON
    reviews = requests.get(query_url).json()
    # Add a twelve second interval between queries to stay within API query limits
    time.sleep(12)
    # Try and save the reviews to the reviews_list
    try:
        for review in reviews["response"]["docs"]:          # loop through the reviews["response"]["docs"] and append each review to the list
            reviews_list.append(review)
        print(f"Retrieved page {page}")                     # Print the page that was just retrieved
    except:
        print (f"No results found on this page: {page}")    # Print the page number that had no results then break from the loop
            


       
        


Retrieved page 0
Retrieved page 1
Retrieved page 2


In [82]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data
##########################This isn't an this weeks activity, but we have looked at the first five elements of a list before###
first_five = reviews_list[:5]
format_five=json.dumps(first_five, indent=4)
print(format_five)


[
    {
        "web_url": "https://www.nytimes.com/2023/05/25/movies/the-attachment-diaries-review.html",
        "snippet": "A gynecologist and her patient form a horrifyingly twisted connection in this batty, bloody Argentine melodrama.",
        "source": "The New York Times",
        "headline": {
            "main": "\u2018The Attachment Diaries\u2019 Review: Love, Sick",
            "kicker": null,
            "content_kicker": null,
            "print_headline": "The Attachment Diaries",
            "name": null,
            "seo": null,
            "sub": null
        },
        "keywords": [
            {
                "name": "subject",
                "value": "Movies",
                "rank": 1,
                "major": "N"
            },
            {
                "name": "creative_works",
                "value": "The Attachment Diaries (Movie)",
                "rank": 2,
                "major": "N"
            },
            {
                "name": "persons",
 

In [83]:
# Convert reviews_list to a Pandas DataFrame using json_normalize()
############################################################Day 1 Activity 8####  (Day 3 Activity 1)
nyt_reviews_df = pd.DataFrame(reviews_list)
nyt_reviews_normalized_df = pd.json_normalize(reviews_list)
nyt_reviews_normalized_df.head()
nyt_reviews_df = nyt_reviews_normalized_df
nyt_reviews_df.head(5)


,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None
1,https://www.nytimes.com/2023/05/04/movies/what...,Two childhood friends navigate cultural differ...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T17:16:45+0000,287,Review: ‘What’s Love Got to Do With It?’ Proba...,None,None,What’s Love Got to Do With It?,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None
2,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None
3,https://www.nytimes.com/2023/04/21/movies/a-to...,Rachael Leigh Cook stars in this bland rom-com...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-21T07:03:25+0000,276,‘A Tourist’s Guide to Love’ Review: A Wearying...,None,None,A Tourist’s Guide to Love,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None
4,https://www.nytimes.com/2023/04/20/movies/othe...,A radiant Virginie Efira stars as a Parisian t...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-20T15:35:13+0000,801,‘Other People’s Children’ Review: True Romance,Critic’s pick,None,Intoxicating Love With a Sobering Turn,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None


In [84]:
# Extract the title from the "headline.main" column and
# save it to a new column "title"
# Title is between unicode characters \u2018 and \u2019. 
# End string should include " Review" to avoid cutting title early
#################This pretty tough, the syntax is in Pdas Day 2 Activity 5########
################# basically, your are going to make a lambda file that will slice the headline.main string####
################# by slice i mean [<startpoint>:<endpoint>]

# Module directions gave this code: (lambda st: st[st.find("\u2018")+1:st.find("\u2019 Review")  ##You could try with regex

nyt_reviews_df["Title"] = nyt_reviews_df["headline.main"].apply(lambda st: st[st.find("\u2018")+1:st.find("\u2019 Review")])
nyt_reviews_df.head()

,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization,Title
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,The Attachment Diaries
1,https://www.nytimes.com/2023/05/04/movies/what...,Two childhood friends navigate cultural differ...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T17:16:45+0000,287,Review: ‘What’s Love Got to Do With It?’ Proba...,None,None,What’s Love Got to Do With It?,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,What’s Love Got to Do With It?’ Probably a Lo
2,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None,You Can Live Forever
3,https://www.nytimes.com/2023/04/21/movies/a-to...,Rachael Leigh Cook stars in this bland rom-com...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-21T07:03:25+0000,276,‘A Tourist’s Guide to Love’ Review: A Wearying...,None,None,A Tourist’s Guide to Love,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None,A Tourist’s Guide to Love
4,https://www.nytimes.com/2023/04/20/movies/othe...,A radiant Virginie Efira stars as a Parisian t...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-20T15:35:13+0000,801,‘Other People’s Children’ Review: True Romance,Critic’s pick,None,Intoxicating Love With a Sobering Turn,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None,Other People’s Children


In [85]:
# Extract 'name' and 'value' from items in "keywords" column
def extract_keywords(keyword_list):
    if keyword_list and isinstance(keyword_list, list):
        extracted_keywords = ""
        for item in keyword_list:
            # Extract 'name' and 'value'
            if "name" in item and "value" in item:
                keyword = f"{item['name']}: {item['value']};" 
                # Append the keyword item to the extracted_keywords list
                extracted_keywords += keyword
        return extracted_keywords      


# Fix the "keywords" column by converting cells from a list to a string
##################you just applying the provided function to the keywords column
nyt_reviews_df["keywords"] = nyt_reviews_df ["keywords"].apply(extract_keywords)
nyt_reviews_df.head()

,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization,Title
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,subject: Movies;creative_works: The Attachment...,2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,The Attachment Diaries
1,https://www.nytimes.com/2023/05/04/movies/what...,Two childhood friends navigate cultural differ...,The New York Times,"subject: Movies;persons: Kapur, Shekhar;person...",2023-05-04T17:16:45+0000,287,Review: ‘What’s Love Got to Do With It?’ Proba...,None,None,What’s Love Got to Do With It?,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,What’s Love Got to Do With It?’ Probably a Lo
2,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,subject: Movies;creative_works: You Can Live F...,2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None,You Can Live Forever
3,https://www.nytimes.com/2023/04/21/movies/a-to...,Rachael Leigh Cook stars in this bland rom-com...,The New York Times,subject: Movies;creative_works: A Tourist's Gu...,2023-04-21T07:03:25+0000,276,‘A Tourist’s Guide to Love’ Review: A Wearying...,None,None,A Tourist’s Guide to Love,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None,A Tourist’s Guide to Love
4,https://www.nytimes.com/2023/04/20/movies/othe...,A radiant Virginie Efira stars as a Parisian t...,The New York Times,"subject: Movies;persons: Zlotowski, Rebecca;cr...",2023-04-20T15:35:13+0000,801,‘Other People’s Children’ Review: True Romance,Critic’s pick,None,Intoxicating Love With a Sobering Turn,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None,Other People’s Children


In [92]:
# Create a list from the "title" column using to_list()
# These titles will be used in the query for The Movie Database
title_list = nyt_reviews_df["Title"].to_list()
title_list


['The Attachment Diaries',
 'What’s Love Got to Do With It?’ Probably a Lo',
 'You Can Live Forever',
 'A Tourist’s Guide to Love',
 'Other People’s Children',
 'One True Loves',
 'The Lost Weekend: A Love Story',
 'A Thousand and One',
 'Your Place or Mine',
 'Love in the Time of Fentanyl',
 'The Attachment Diaries',
 'What’s Love Got to Do With It?’ Probably a Lo',
 'You Can Live Forever',
 'A Tourist’s Guide to Love',
 'Other People’s Children',
 'One True Loves',
 'The Lost Weekend: A Love Story',
 'A Thousand and One',
 'Your Place or Mine',
 'Love in the Time of Fentanyl',
 'The Attachment Diaries',
 'What’s Love Got to Do With It?’ Probably a Lo',
 'You Can Live Forever',
 'A Tourist’s Guide to Love',
 'Other People’s Children',
 'One True Loves',
 'The Lost Weekend: A Love Story',
 'A Thousand and One',
 'Your Place or Mine',
 'Love in the Time of Fentanyl']

### Access The Movie Database API

In [101]:
# Prepare The Movie Database query
url = "https://api.themoviedb.org/3/search/movie?query="
tmdb_key_string = "&api_key=" + tmdb_api_key

In [128]:
# Create an empty list to store the results

tmdb_movies_list = []

# Create a request counter to sleep the requests after a multiple of 50 requests  (##Mods or Python Modules)
request_counter = 1

# Loop through the titles
for title in title_list:      
        # Check if we need to sleep before making a request * 50 request
        request_counter += 1
        if request_counter % 50 == 0:
                print(f"Sleeping at {request_counter} requests...")
                time.sleep(1)  #sleep for 1 second (adjustable)

        try:
                request_url = f"{url}{title}{tmdb_key_string}"
                response = requests.get(request_url).json()
                try:
                        movie_id = response["results"][0]["id"]
                        movie_query= f"https://api.themoviedb.org/3/movie/{movie_id}?{tmdb_key_string}"
                        movie_details = requests.get(movie_query).json()                
                        
                        # Extract the genre names into a list  
                        genres = [genre["name"] for genre in movie_details.get("genres", [])]
                        # Extract the 'spoken_languages' into a list
                        spoken_languages = [lang["english_name"] for lang in movie_details.get("spoken_languages", [])]
                        # # Extract the 'production_countries' name into a list
                        # production_countries = [pro_co["specific_country"] for pro_co in movie_details.get("production_countries",)]

                        #Add the relevant data to a dictionary and # append it to the tmdb_movies_list list
                        movie_info= {
                                "title": movie_details.get("title"),
                                "original_title": movie_details.get("original_title"),
                                "budget": movie_details.get("budget", 0),
                                "original_language": movie_details.get("original_language"),
                                "home_page": movie_details.get("home_page"),
                                "overview": movie_details.get("overview"),
                                "popularity":movie_details.get("popularity"),
                                "runtime":movie_details.get("runtime"),
                                "revenue":movie_details.get("revenue"),
                                "release_date":movie_details.get("release_date"),
                                "vote_average":movie_details.get("vote_average"),
                                "revenue":movie_details.get("revenue"),
                                "vote_average": movie_details.get("vote_average"),
                                "genres": genres,
                                "spoken_languages":spoken_languages,
                                # "production_countries": production_countries
                                }

                        tmdb_movies_list.append(movie_info)
                        print(f"Found: {title}")
                except Exception as e:
                        print(f"The first error occured. Error message: {str(e)}")
        except Exception as e:
                print(f"An error occured. Error message: {str(e)}")
  
tmdb_movies_list

Found: The Attachment Diaries
The first error occured. Error message: list index out of range
Found: You Can Live Forever
Found: A Tourist’s Guide to Love
Found: Other People’s Children
Found: One True Loves
Found: The Lost Weekend: A Love Story
Found: A Thousand and One
Found: Your Place or Mine
Found: Love in the Time of Fentanyl
Found: The Attachment Diaries
The first error occured. Error message: list index out of range
Found: You Can Live Forever
Found: A Tourist’s Guide to Love
Found: Other People’s Children
Found: One True Loves
Found: The Lost Weekend: A Love Story
Found: A Thousand and One
Found: Your Place or Mine
Found: Love in the Time of Fentanyl
Found: The Attachment Diaries
The first error occured. Error message: list index out of range
Found: You Can Live Forever
Found: A Tourist’s Guide to Love
Found: Other People’s Children
Found: One True Loves
Found: The Lost Weekend: A Love Story
Found: A Thousand and One
Found: Your Place or Mine
Found: Love in the Time of Fentany

[{'title': 'The Attachment Diaries',
  'original_title': 'El apego',
  'budget': 0,
  'original_language': 'es',
  'home_page': None,
  'overview': 'Argentina, 1970s. A desperate young woman goes to a clinic to have a clandestine abortion. As her pregnancy is already through the fourth month, the doctor refuses. Instead, she proposes to sell the baby to one of her clients and offers to provide shelter in her house until the child is born. Their disturbed personalities will become intertwined in a strange and dangerous relationship.',
  'popularity': 1.26,
  'runtime': 102,
  'revenue': 0,
  'release_date': '2021-10-07',
  'vote_average': 4.0,
  'genres': ['Drama', 'Mystery', 'Thriller', 'Horror'],
  'spoken_languages': ['Spanish']},
 {'title': 'You Can Live Forever',
  'original_title': 'You Can Live Forever',
  'budget': 0,
  'original_language': 'en',
  'home_page': None,
  'overview': "When Jaime, a gay teenager, is sent to live in a community of Jehovah's Witnesses in small-town Qu

In [122]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data
print(json.dumps(tmdb_movies_list, indent=4))


[
    {
        "title": "The Attachment Diaries",
        "original_title": "El apego",
        "budget": 0,
        "original_language": "es",
        "home_page": null,
        "overview": "Argentina, 1970s. A desperate young woman goes to a clinic to have a clandestine abortion. As her pregnancy is already through the fourth month, the doctor refuses. Instead, she proposes to sell the baby to one of her clients and offers to provide shelter in her house until the child is born. Their disturbed personalities will become intertwined in a strange and dangerous relationship.",
        "popularity": 1.26,
        "runtime": 102,
        "revenue": 0,
        "release_date": "2021-10-07",
        "vote_average": 4.0,
        "genres": [
            "Drama",
            "Mystery",
            "Thriller",
            "Horror"
        ],
        "spoken_languages": [
            "Spanish"
        ]
    },
    {
        "title": "You Can Live Forever",
        "original_title": "You Can Live

In [124]:
# Convert the results to a DataFrame
movies_df=pd.DataFrame(tmdb_movies_list)
movies_df

,title,original_title,budget,original_language,home_page,overview,popularity,runtime,revenue,release_date,vote_average,genres,spoken_languages
0,The Attachment Diaries,El apego,0,es,None,"Argentina, 1970s. A desperate young woman goes...",1.260,102,0,2021-10-07,4.000,"[Drama, Mystery, Thriller, Horror]",[Spanish]
1,You Can Live Forever,You Can Live Forever,0,en,None,"When Jaime, a gay teenager, is sent to live in...",7.338,96,0,2022-06-11,7.263,"[Drama, Romance]","[French, English]"
2,A Tourist's Guide to Love,A Tourist's Guide to Love,0,en,None,"After an unexpected break up, a travel executi...",14.960,96,0,2023-04-21,6.388,"[Romance, Comedy, Adventure]",[English]
3,Other People's Children,Les Enfants des autres,0,fr,None,"Rachel loves her life, her students, her frien...",10.423,104,0,2022-09-21,6.917,[Drama],[French]
4,One True Loves,One True Loves,0,en,None,Emma and Jesse are living the perfect life tog...,13.103,100,37820,2023-04-07,6.761,"[Romance, Drama, Comedy]",[English]
5,The Lost Weekend: A Love Story,The Lost Weekend: A Love Story,0,en,None,May Pang lovingly recounts her life in rock & ...,2.071,95,0,2023-04-13,6.000,[Documentary],[English]
6,A Thousand and One,A Thousand and One,0,en,None,Struggling but unapologetically living on her ...,16.573,116,0,2023-03-31,7.021,[Drama],[English]
7,Your Place or Mine,Your Place or Mine,0,en,None,When best friends and total opposites Debbie a...,23.250,109,0,2023-02-10,6.309,"[Romance, Comedy]",[English]
8,Love in the Time of Fentanyl,Love in the Time of Fentanyl,0,en,None,An intimate portrait of a community fighting t...,2.339,85,0,2023-02-03,0.000,[Documentary],[English]
9,The Attachment Diaries,El apego,0,es,None,"Argentina, 1970s. A desperate young woman goes...",1.260,102,0,2021-10-07,4.000,"[Drama, Mystery, Thriller, Horror]",[Spanish]


### Merge and Clean the Data for Export

In [ ]:
# Merge the New York Times reviews and TMDB DataFrames on title
## IT IS A MERGE!!!!!
## GET RID OF EXTRA BRACKETS AND QUOTE.  

In [ ]:
# Remove list brackets and quotation marks on the columns containing lists
# Create a list of the columns that need fixing
##THREE columns have extra list and extra quotes...

# Create a list of characters to remove


# Loop through the list of columns to fix

    # Convert the column to type 'str'


    # Loop through characters to remove


# Display the fixed DataFrame


In [ ]:
# Drop "byline.person" column
##PANDAS

In [ ]:
# Delete duplicate rows and reset index
## drop is part of code

In [ ]:
# Export data to CSV without the index
